In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def get_elo_of_players_in_match(df,matchnr):
    getmatchID = df.matchId[matchnr]
    eloLIstrank  = df.rankPoints[df['matchId'] == getmatchID]
    eloListWin = df.winPoints[df['matchId'] == getmatchID]
    eloLIstrank = np.sort(eloLIstrank)
    eloListWin = np.sort(eloListWin)
    return  eloLIstrank,eloListWin
def create_cat_values(dataframe,column):
    dataframe[column] = dataframe[column].astype('category')
    dataframe[column] = dataframe[column].cat.codes
    column_lenght = len(dataframe[column].unique())
    print(column + ' columnContains:' + str(column_lenght) + ' categories')
    return dataframe

In [ ]:
df = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')

In [ ]:
df.columns

In [ ]:
print(get_elo_of_players_in_match(df,20))
get_elo_of_players_in_match(df,21)

In [ ]:
df.head()

In [ ]:
df.rankPoints[df['matchId'] == 'd5f02b7d5ed782']

In [ ]:
create_cat_values(df,'Id')
create_cat_values(df,'groupId')
create_cat_values(df,'matchId')

In [ ]:
x_train,y_train=df.iloc[:,0:-1],df.iloc[:,-1]
print(x_train)
y_train

In [ ]:
solo = df[df.matchType == 'solo']
duo = df[df.matchType == 'duo']
squad = df[df.matchType == 'squad']
solofpp = df[df.matchType == 'solo-fpp']
duofpp = df[df.matchType == 'duo-fpp']
squadfpp = df[df.matchType == 'squad-fpp']
print('Solo subset size ' + str(len(solo)))
print('Duo subset size ' + str(len(duo)))
print('Squad subset size ' + str(len(squad)))
print('SoloFPP subset size ' + str(len(solofpp)))
print('DuoFPP subset size '+ str(len(duofpp)))
print('SquadFPP subset size '+ str(len(squadfpp)))

In [ ]:
squadfpp

In [ ]:
test = squadfpp.groupby(['groupId','matchId']).count()
test

In [ ]:
test[test['Id']> 4]

In [ ]:
test[test['Id']> 30]

In [ ]:
squadfpp[squadfpp['groupId']== 1297474]

In [ ]:
solofirst = solofpp[['Id','groupId']].groupby('groupId').count().max()[0]
solothird = solo[['Id','groupId']].groupby('groupId').count().max()[0]
squadfirst = squadfpp[['Id','groupId']].groupby('groupId').count().max()[0]
squadthird = squad[['Id','groupId']].groupby('groupId').count().max()[0]
duofirst = duofpp[['Id','groupId']].groupby('groupId').count().max()[0]
duothird = duo[['Id','groupId']].groupby('groupId').count().max()[0]
print('Solo max group FP:' + str(solofirst) + " TP:" + str(solothird) )
print('Squad max group FP:' + str(squadfirst) + " TP:" +str(squadthird))
print('Duo max group FP:' + str(duofirst) + " TP:" +str(duothird))

In [ ]:
listofcount = solofpp[['matchId','groupId','Id']].groupby(['groupId','matchId']).count()
listofcount.Id >2 

In [ ]:
solot = 1
duot = 2
squadt = 4

def get_all_matches(df,maxplayer):
    matcharr = []
    listofcount = df[['matchId','groupId','Id']].groupby(['groupId','matchId']).count()
    listofcount = listofcount[listofcount['Id'] > solot].index
    i = 0
    for items in listofcount:
        matchid = items[1]
        matcharr.append(matchid)
        i = i + 1
    print("Unique number of games: "+str(len(df['matchId'].unique()))+" Unique number of games with more than the acceptable groupnumbers: "+str(len(np.unique(matcharr))))
    matcharr = np.unique(matcharr)
    return matcharr
solofppmatchestodrop = get_all_matches(solofpp,solot)
solomatchestodrop = get_all_matches(solo,solot)
get_all_matches(duofpp,duot)
get_all_matches(duo,duot)
get_all_matches(squadfpp,squadt)
get_all_matches(squad,squadt)

In [ ]:
len(solofppmatchestodrop)

In [ ]:
lenbefore = len(solofpp)
for items in solofppmatchestodrop:
    solofpp = solofpp[solofpp.matchId != items]
lenafter = len(solofpp)
print('Lenght of data before dropping all wrong group measures: ' + str(lenbefore) + " Len after: " + str(lenafter))

lenbefore = len(solo)
for items in solomatchestodrop:
    solo = solo[solo.matchId != items]
lenafter = len(solo)
print('Lenght of data before dropping all wrong group measures: ' + str(lenbefore) + " Len after: " + str(lenafter))

In [ ]:
print(len(solofpp['matchId'].unique()))
len(solo['matchId'].unique())

In [ ]:
solofpp.columns
solofpp = solofpp[['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints','rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc']]
solo.columns
solo = solo[['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints','rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc']]

In [ ]:
solofpp

In [ ]:
import matplotlib.pyplot as plt
def createplots(df):
    coltoplot = ['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc']
    for columns in coltoplot:
        plt.scatter(df.winPlacePerc, df[columns], alpha=0.5)
        plt.title(columns)
        plt.xlabel('Placement')
        plt.ylabel(columns)
        plt.show()

In [ ]:
createplots(solofpp)

In [ ]:
solofpp.columns

In [ ]:
boxplot = solofpp.boxplot(['winPlacePerc'])
solofpp

In [ ]:
solofpp.columns
col = ['damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration','winPlacePerc']#,
       #'rideDistance', 'roadKills',
       #'swimDistance', 'vehicleDestroys', 'walkDistance',
       #'weaponsAcquired']

In [ ]:
solofpp.nunique()

In [ ]:
from sklearn.model_selection import train_test_split
solofpp = solofpp.dropna()
solofpp.isnull().sum()
y_train = solofpp['winPlacePerc']
X_train = solofpp[['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']]
X_train = X_train[['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
###############################
#           Validate          #
###############################
y_val = solo['winPlacePerc']
X_val = solo[['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
print(solofpp.dtypes)

In [ ]:
X_train.count()

In [ ]:
import matplotlib.pyplot as plt
for items in X_train.columns:
    plt.figure()
    X_train.boxplot([items])

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(300, activation="relu", name="Input_layer"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(80, activation="relu"),
        tf.keras.layers.Dense(40, activation="relu"),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(20, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss='mse',
    # List of metrics to monitor
    metrics=['MSE'],
)
history  = model.fit(X_train,y_train,epochs = 200)

In [ ]:
networkoutput = model.predict(X_train)

In [ ]:
prediction = y_train.values

In [ ]:
df_to_test_difference = pd.DataFrame(networkoutput,columns = ['Prediction'])
df_to_test_difference['Actual'] = prediction
df_to_test_difference['Difference'] = df_to_test_difference['Actual']- df_to_test_difference['Prediction']

In [ ]:
df_to_test_difference

In [ ]:
print("Average deviation: ")
print(df_to_test_difference['Difference'].mean())
print('Maximum deviation: ')
print(df_to_test_difference['Difference'].max())
print('Minimum deviation: ')
print(df_to_test_difference['Difference'].min())

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df_to_test_difference.Prediction,df_to_test_difference.Actual)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
import seaborn as sns
ax = sns.boxplot(x=df_to_test_difference["Difference"])

In [ ]:
val_prediction = model.predict(X_val)
# Create a dataframe to hold the predicted, actual and difference between the two.
df_to_val_difference = pd.DataFrame(val_prediction,columns = ['Prediction'])
df_to_val_difference['Actual'] = y_val.values
df_to_val_difference['Difference'] = df_to_val_difference['Actual']- df_to_val_difference['Prediction']
df_to_val_difference

In [ ]:
print("Average deviation: ")
print(df_to_val_difference['Difference'].mean())
print('Maximum deviation: ')
print(df_to_val_difference['Difference'].max())
print('Minimum deviation: ')
print(df_to_val_difference['Difference'].min())

In [ ]:
colls = ['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints','rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']
rawtest = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')
tesetdf = rawtest
solotest = tesetdf[df.matchType == 'solo']
duotest = tesetdf[tesetdf.matchType == 'duo']
squadtest = tesetdf[tesetdf.matchType == 'squad']
solofpptest = tesetdf[tesetdf.matchType == 'solo-fpp']
duofpptest = tesetdf[tesetdf.matchType == 'duo-fpp']
squadfpptest = tesetdf[tesetdf.matchType == 'squad-fpp']

In [ ]:
solotestmatchestodrop = get_all_matches(solotest,solot)
solofpptestmatchestodrop = get_all_matches(solofpptest,solot)
get_all_matches(duotest,duot)
get_all_matches(duofpptest,duot)
get_all_matches(squadtest,squadt)
get_all_matches(squadfpptest,squadt)

In [ ]:
lenbefore = len(solofpptest)
for items in solofpptestmatchestodrop:
    solofpptest = solofpptest[solofpptest.matchId != items]
lenafter = len(solofpptest)
print('Lenght of data before dropping all wrong group measures: ' + str(lenbefore) + " Len after: " + str(lenafter))

lenbefore = len(solotest)
for items in solotestmatchestodrop:
    solotest = solotest[solotest.matchId != items]
lenafter = len(solotest)
print('Lenght of data before dropping all wrong group measures: ' + str(lenbefore) + " Len after: " + str(lenafter))

In [ ]:

solotest = solotest[colls].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
solofpptest = solofpptest[colls].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
tesetdf = tesetdf[colls].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
testoutput = model.predict(solotest)
solotest['Id'] = Idtouse
solotest['winPlacePerc'] = testoutput

In [ ]:
solotest[['Id','winPlacePerc']]

In [ ]:
legal_model_output = model.predict(tesetdf)

In [ ]:
legalmodelsub = rawtest
rawtest['winPlacePerc'] = legal_model_output
submissionlegalmoves = rawtest[['Id','winPlacePerc']]
#submissionlegalmoves
submissionlegalmoves.to_csv("submission.csv",index = False)

In [ ]:
submissionlegalmoves

In [ ]:
submissionlegal_full_games = solotest[['Id','winPlacePerc']]
submissionlegal_full_games

In [ ]:
submissiondf = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv')
submissiondf